In [29]:
def containsN(index):
    return "N" in index

index1 = "NAAAAAAA"
index2 = "TTTTTTTT"
index3 = "GGGGGGGG"

b1 = containsN(index1)
b2 = containsN(index2)
b3 = containsN(index3)

print(b1, b2, b3)
#Expected Results:
print("TRUE", "FALSE", "FALSE")

True False False
TRUE FALSE FALSE


In [30]:
def matching(index1, index2):
    matched = False
    inverse = ""
    
    for x in index2:
        if x == "A":
            inverse =  "T" + inverse 
        elif x == "T":
            inverse =  "A" + inverse
        elif x == "G":
            inverse = "C" + inverse
        elif x == "C":
            inverse = "G" + inverse   
        else:
            inverse = "N" + inverse

#assuming N base calls are normally on the first base
#first 100 reads in both files only have Ns on first base
#this allows one base pair difference between matching indexes
#keeps down the rate of false positives 

    if containsN(index1):
        index1 = index1[1:]
        inverse = inverse[1:]
        
    elif containsN(index2):
        index1 = index1[:-1]
        inverse = inverse[:-1]
        
    elif containsN(index1) and containsN(index2):
        index1 = index1[1:-1]
        inverse = inverse[1:-1]
        
    return (inverse == index1)

T1 =matching("AAAAAAAA", "TTTTTATT")
T2 = matching("AAAAAAAA", "AAAAAAAA")
T3 = matching("AAAAAAAA", "TTTTTTTT")
T4 = matching("NGTTCCGT", "NCGGAACA")
T5 = matching("GATTACAA", "TTGTAATC")
T6 = matching("AAAAAAAA", "TAAAAAAA")

print(T1, T2, T3, T4, T5, T6)
#Expected Results:
print("FALSE", "FALSE", "TRUE", "TRUE", "TRUE", "FALSE")

False False True False True False
FALSE FALSE TRUE TRUE TRUE FALSE


In [31]:
def writeToBadFile(outputFile, read, reason):
    N = 1
    IH = 2
    QS = 4
    E = 8 #error in index
    
    if (reason & N):
        read[2] = read[2] + " " + "(N)"
        
    if (reason & IH):
        read[2] = read[2] + " " + "(IH)"
        
    if (reason & QS):
        read[2] = read[2] + " " + "(QS)"
        
    if (reason & E):
        read[2] = read[2] + " " + "(E)"
        
    with open (outputFile, "a") as out:
        out.write(read[0])
        out.write("\n")
        out.write(read[1])
        out.write("\n")
        out.write(read[2])
        out.write("\n")
        out.write(read[3])
        out.write("\n")
        
outfile = "badFile.txt"

read1 = ["@headerline", "CATACTGGTCA" , "+ NAAAAAAA", "DDDDDDDDDDD"]
read2 = ["@headerline2", "CATACTCAGTA", "+ TTTTTTTT", "JJJJJJJJJJJ"]
read3 = ["@headerline3", "ACGCTGCGATC", "+ GGGGGGGG", "HHHHHHHHHHH"]

writeToBadFile(outfile, read1, 1)
writeToBadFile(outfile, read2, 2)
writeToBadFile(outfile, read3, 4)
writeToBadFile(outfile, read1, 3)
writeToBadFile(outfile, read2, 5)
writeToBadFile(outfile, read3, 6)
writeToBadFile(outfile, read3, 7)

#arrays are passed by reference so the flags are there for the previous reads
#This won't have an affect on uniq reads in the case of the script

In [32]:
def writeToGoodFile(outputFile, read):
    with open (outputFile, "a") as out:
        out.write(read[0])
        out.write("\n")
        out.write(read[1])
        out.write("\n")
        out.write(read[2])
        out.write("\n")
        out.write(read[3])
        out.write("\n")
        
outfile = "goodFile.txt"

read1 = ["@headerline", "CATACTGGTCA" , "+ NAAAAAAA", "DDDDDDDDDDD"]
read2 = ["@headerline2", "CATACTCAGTA", "+ TTTTTTTT", "JJJJJJJJJJJ"]
read3 = ["@headerline3", "ACGCTGCGATC", "+ GGGGGGGG", "HHHHHHHHHHH"]

writeToGoodFile(outfile, read1)
writeToGoodFile(outfile, read2)
writeToGoodFile(outfile, read3)        
        
outfile = "goodFile2.txt"

read1 = ["@headerline", "CATACTGGTCA" , "+ NAAAAAAA", "DDDDDDDDDDD"]
read2 = ["@headerline2", "CATACTCAGTA", "+ TTTTTTTT", "JJJJJJJJJJJ"]
read3 = ["@headerline3", "ACGCTGCGATC", "+ GGGGGGGG", "HHHHHHHHHHH"]

writeToGoodFile(outfile, read1)
writeToGoodFile(outfile, read2)
writeToGoodFile(outfile, read3)

#file was written and all looks good.

In [33]:
def convertPhred(letter):
    return ord(letter)-33

phred1 = convertPhred("A")
phred2 = convertPhred("#")
phred3 = convertPhred("I")
phred4 = convertPhred("J")

print(phred1, phred2, phred3, phred4)
#Expected Results:
print(32, 2, 40, 41)

32 2 40 41
32 2 40 41


In [34]:
def meanQS(read):
    total = 0
    bp = 0
    for base in read[3]:
        total = total + convertPhred(base)
        bp = bp + 1
    return total/bp

read1 = ["@headerline", "CATACTGGTCA" , "+ NAAAAAAA", "###########"]
read2 = ["@headerline2", "CATACTCAGTA", "+ TTTTTTTT", "JJJJJJJJJJJ"]
read3 = ["@headerline3", "ACGCTGCGATC", "+ GGGGGGGG", "AAAAAAAAAAA"]

QS1 = meanQS(read1)
QS2 = meanQS(read2)
QS3 = meanQS(read3)

print(QS1, QS2, QS3)
#Expected Results
print(2, 41, 32)

2.0 41.0 32.0
2 41 32


In [35]:
def compareMeans(read):
    cutoffQS = 32
    return meanQS(read) >= cutoffQS

read1 = ["@headerline", "CATACTGGTCA" , "+ NAAAAAAA", "###########"]
read2 = ["@headerline2", "CATACTCAGTA", "+ TTTTTTTT", "JJJJJJJJJJJ"]
read3 = ["@headerline3", "ACGCTGCGATC", "+ GGGGGGGG", "AAAAAAAAAAA"]

QS1 = compareMeans(read1)
QS2 = compareMeans(read2)
QS3 = compareMeans(read3)

print(QS1, QS2, QS3)
#Expected Results:
print("FALSE", "TRUE", "TRUE")

False True True
FALSE TRUE TRUE


In [36]:
def readInLine(file):
    line = file.readline()
    return line.strip()

file1 = "goodFile.txt"
file2 = "goodFile2.txt"

fh1 = open(file1, "r")
fh2 = open(file2, "r")

for x in range(12):  
    line = readInLine(fh1)
    print(line, 1)
    line2 = readInLine(fh2)
    print(line2)
    
#the same line of the files are run if they are in the same loop

@headerline 1
@headerline
CATACTGGTCA 1
CATACTGGTCA
+ NAAAAAAA 1
+ NAAAAAAA
DDDDDDDDDDD 1
DDDDDDDDDDD
@headerline2 1
@headerline2
CATACTCAGTA 1
CATACTCAGTA
+ TTTTTTTT 1
+ TTTTTTTT
JJJJJJJJJJJ 1
JJJJJJJJJJJ
@headerline3 1
@headerline3
ACGCTGCGATC 1
ACGCTGCGATC
+ GGGGGGGG 1
+ GGGGGGGG
HHHHHHHHHHH 1
HHHHHHHHHHH
